In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

import pandas as pd
from collections import OrderedDict

import pprint
import re
import numpy as np
import matplotlib.pyplot as plt
import random

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
init_notebook_mode(connected=True)  
import glob
import uproot 
import ROOT

import platform
print('Python:', platform.python_version())

Welcome to JupyROOT 6.26/06
Python: 3.10.8


In [2]:
# run = 10645
# run = 10647
# run = 10648
#  filenames = glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc*_pedestal-*{run}-0000.prdf.root")

# run = 10661
# filenames = glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc*_pedestal-*{run}-0000.prdf_1.root")

run = 10684
filenames = glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc*_beam-*{run}-0000.prdf_skip500.root")

filenames.sort()
pprint.pprint(filenames)

['/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc00_beam-00010684-0000.prdf_skip500.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc01_beam-00010684-0000.prdf_skip500.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc02_beam-00010684-0000.prdf_skip500.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc03_beam-00010684-0000.prdf_skip500.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc04_beam-00010684-0000.prdf_skip500.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc05_beam-00010684-0000.prdf_skip500.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc06_beam-00010684-0000.prdf_skip500.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc07_beam-00010684-0000.prdf_skip500.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc08_beam-00010684-0000.prdf_skip500.root',
 '/phenix/u/jinhuang/links/s

In [3]:
files = dict()
for filename in filenames: 
    file = ROOT.TFile.Open(filename)
    sector = re.match(".*ebdc([0-9][0-9])_.*", filename).group(1)
    files[sector] = file


pprint.pprint(files);

{'00': <cppyy.gbl.TFile object at 0x84386e0>,
 '01': <cppyy.gbl.TFile object at 0x6b4e320>,
 '02': <cppyy.gbl.TFile object at 0x876bd30>,
 '03': <cppyy.gbl.TFile object at 0x8715ed0>,
 '04': <cppyy.gbl.TFile object at 0x858ef90>,
 '05': <cppyy.gbl.TFile object at 0x8832420>,
 '06': <cppyy.gbl.TFile object at 0x87c89d0>,
 '07': <cppyy.gbl.TFile object at 0x8726470>,
 '08': <cppyy.gbl.TFile object at 0x883f1e0>,
 '09': <cppyy.gbl.TFile object at 0x8841dd0>,
 '10': <cppyy.gbl.TFile object at 0x8844af0>,
 '11': <cppyy.gbl.TFile object at 0x88467c0>,
 '12': <cppyy.gbl.TFile object at 0x888e670>,
 '13': <cppyy.gbl.TFile object at 0x88912b0>,
 '14': <cppyy.gbl.TFile object at 0x8893f50>,
 '15': <cppyy.gbl.TFile object at 0x88a6dc0>,
 '16': <cppyy.gbl.TFile object at 0x88a9ae0>,
 '17': <cppyy.gbl.TFile object at 0x88ac7e0>,
 '18': <cppyy.gbl.TFile object at 0x88af580>,
 '19': <cppyy.gbl.TFile object at 0x88b2270>,
 '20': <cppyy.gbl.TFile object at 0x88b3c00>,
 '21': <cppyy.gbl.TFile object at 

In [4]:
files['02'].ls()
files['02'].PacketTree.Show(0)
files['02'].SampleTree.Show(0)
files['02'].TaggerTree.Show(0)

TFile**		/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc02_beam-00010684-0000.prdf_skip500.root	
 TFile*		/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc02_beam-00010684-0000.prdf_skip500.root	
  KEY: TTree	SampleTree;47	Each entry is one waveform [current cycle]
  KEY: TTree	SampleTree;46	Each entry is one waveform [backup cycle]
  KEY: TTree	PacketTree;1	Each entry is one packet
  KEY: TTree	TaggerTree;1	Each entry is one tagger for level 1 trigger or endat tag
======> EVENT:0
 packet          = 4020
 frame           = 2
 nWaveormInFrame = 663
 nTaggerInFrame  = 2
 maxFEECount     = 26
======> EVENT:0
 packet          = 4020
 frame           = 2
 nWaveormInFrame = 663
 maxFEECount     = 26
 nSamples        = 360
 adcSamples      = 75, 
                  78, 68, 70, 74, 72, 72, 75, 72, 70, 70, 
                  76, 73, 72, 70, 70, 82, 70, 71, 74
 fee             = 22
 sampaAddress    = 6
 sampaChannel    = 17
 Channel         = 209
 BCO             =

In [5]:
%%cpp -d

#include<sPhenixStyle.C>

In [6]:
%%cpp

SetsPhenixStyle()

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


In [7]:
%jsroot on

In [8]:
c1 = ROOT.TCanvas( 'checksumErrorQA', 'checksumErrorQA', 1000, 1200)

c1.Divide(4,6)

i = 1
for sector, file in files.items():
    
    c1.cd(i)
    i=i+1
    
    hAllWaveForm = ROOT.TH1F(f'hAllWaveForm{sector}', ';fee*8+sampaAddress [0..207]', 26*8, -.5, 26*8 -.5);
    hErrorWaveForm = ROOT.TH1F(f'hErrorWaveForm{sector}', f';Sector{sector}: fee*8+sampaAddress [0..207];Probability of CheckSum Error', 26*8, -.5, 26*8 -.5);

    files[sector].SampleTree.Draw(f"fee*8+sampaAddress >> hAllWaveForm{sector}","","goff");
    files[sector].SampleTree.Draw(f"fee*8+sampaAddress >> hErrorWaveForm{sector}","checksumError>0","goff");
    
    hErrorWaveForm.Divide (hErrorWaveForm, hAllWaveForm, 1, 1, 'B');
    hErrorWaveForm.DrawClone();

c1.Draw()

In [9]:

c1 = ROOT.TCanvas( 'ADCQA', 'ADCQA', 1000, 1200)

c1.Divide(4,6)

i = 1
for sector, file in files.items():
    

    pad = c1.cd(i)
    i=i+1

    pad.SetLogz()
    
    hADC = ROOT.TH2F(f'hADC{sector}', f';Sector{sector}: ADC Time bin [1/20MHz]; ADC [ADU]', 360, 0, 360, 256,0,1024);

    files[sector].SampleTree.Draw(f"adcSamples:Iteration$>>hADC{sector}","checksumError==0","colz");
    
    hADC.DrawClone("colz");
    
c1.Draw()